# Version IX - Ensemble GeometryOperations (10 models, Geometry Operations Data Augmentation)

### Imports and initial setup

In [1]:
import import_ipynb

from InitialSetup import dataset, dataset_length, BATCH_SIZE, tf, prepare_callbacks, model_VI, show_history, test_dataset, test_dataset_length, AUTOTUNE, train_models, load_weights, load_weights_read, evaluate_models, get_labels_logits_and_preds, get_class_preds, get_class_from_sum_of_logits, np, BatchNormalization, LeakyReLU, process_rotate, process_shear, process_translate, process_crop

NUM_MODELS = 10

importing Jupyter notebook from InitialSetup.ipynb
Num GPUs Available:  1
Total images in validatation dataset:  12630
(32, 32, 3) (43,)
Total images in dataset:  39209


### Prepare dataset

In [4]:
BATCH_SIZE = 64

dataSolo = dataset
dataSolo = dataSolo.concatenate(dataset.map(process_rotate))
dataSolo = dataSolo.concatenate(dataset.map(process_shear))
dataSolo = dataSolo.concatenate(dataset.map(process_translate))
dataSolo = dataSolo.concatenate(dataset.map(process_crop))

dataSolo_size = tf.data.experimental.cardinality(dataSolo).numpy()

dataSolo = dataSolo.cache()
dataSolo = dataSolo.shuffle(buffer_size = dataSolo_size)
dataSolo = dataSolo.batch(batch_size = BATCH_SIZE)
dataSolo = dataSolo.prefetch(buffer_size = AUTOTUNE)

train_size = int(0.8* dataSolo_size)
val_size = int(0.2* dataSolo_size)

train_dataset = dataSolo.take(train_size)
val_dataset = dataSolo.skip(train_size)

file_path_prefix = './Networks/ensembles/V3/V3'    

### Setup models and save folder

In [3]:
models_V2, histories_V2 = train_models(dataSolo, train_size/BATCH_SIZE, val_dataset, val_size/BATCH_SIZE, file_path_prefix, NUM_MODELS)

Epoch 1/20
2450/2450 [==============================] - 132s 29ms/step - loss: 2.4509 - accuracy: 0.4288 - val_loss: 0.2842 - val_accuracy: 0.9215

Epoch 00001: val_accuracy improved from -inf to 0.92149, saving model to ./Networks/ensembles/V3/V3_00\cp.ckpt
Epoch 2/20
2450/2450 [==============================] - 71s 29ms/step - loss: 0.4788 - accuracy: 0.8602 - val_loss: 0.1118 - val_accuracy: 0.9677

Epoch 00002: val_accuracy improved from 0.92149 to 0.96765, saving model to ./Networks/ensembles/V3/V3_00\cp.ckpt
Epoch 3/20
2450/2450 [==============================] - 70s 28ms/step - loss: 0.2223 - accuracy: 0.9345 - val_loss: 0.0367 - val_accuracy: 0.9903

Epoch 00003: val_accuracy improved from 0.96765 to 0.99029, saving model to ./Networks/ensembles/V3/V3_00\cp.ckpt
Epoch 4/20
2450/2450 [==============================] - 70s 28ms/step - loss: 0.1229 - accuracy: 0.9626 - val_loss: 0.0182 - val_accuracy: 0.9949

Epoch 00004: val_accuracy improved from 0.99029 to 0.99490, saving model

### Load Weights and evaluate models

In [5]:
models_V2 = load_weights_read(file_path_prefix, NUM_MODELS)
evaluate_models(models_V2, NUM_MODELS)

198/198 - 3s - loss: 0.0420 - accuracy: 0.9883
198/198 - 2s - loss: 0.0393 - accuracy: 0.9886
198/198 - 2s - loss: 0.0360 - accuracy: 0.9907
198/198 - 2s - loss: 0.0394 - accuracy: 0.9882
198/198 - 2s - loss: 0.0570 - accuracy: 0.9834
198/198 - 2s - loss: 0.0575 - accuracy: 0.9851
198/198 - 2s - loss: 0.0468 - accuracy: 0.9886
198/198 - 2s - loss: 0.0563 - accuracy: 0.9875
198/198 - 2s - loss: 0.0369 - accuracy: 0.9877
198/198 - 2s - loss: 0.0439 - accuracy: 0.9889
average accuracy: 98.770


In [6]:
labels_V2, logits_V2, preds_V2 = get_labels_logits_and_preds(models_V2, NUM_MODELS)
class_preds_V2 = get_class_preds(preds_V2, NUM_MODELS)
class_logits_V2 = get_class_from_sum_of_logits(logits_V2, NUM_MODELS)

### Build list of inputs and labels for test class

In [7]:
test_logits_preds = []

for i in range(test_dataset_length):
    
    aux = []   
    for m in range(NUM_MODELS):        
        aux.extend(logits_V2[m][i])
        
    test_logits_preds.append(aux)

### Compute logit predictions and training labels

In [8]:
dataSolo_len = len(dataSolo.take(-1))
j=0

logits_train = [[] for _ in range(NUM_MODELS)]
labels_aux = []
for images, labs in dataSolo.take(dataSolo_len):

    #Printing progress
    print('\r' + f'{j}/{dataSolo_len}',end="")
    j+=1
    
    labels_aux.extend(labs.numpy())
    for i in range(NUM_MODELS):
        
        logits_train[i].extend(models_V2[i][1].predict(images))
        
labels_train = [np.argmax(i) for i in labels_aux]  

3063/3064

### Build list of train inputs

In [9]:
train_logits_preds = []

for i in range(dataSolo_size):
    
    aux = []
    
    for m in range(NUM_MODELS):
        
        aux.extend(logits_train[m][i])
        
    train_logits_preds.append(aux)

In [10]:
print(len(labels_train), len(train_logits_preds))

196045 196045


In [11]:

stack_model  = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(len(train_logits_preds[0]),)),

  tf.keras.layers.Dense(256),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(64),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),

  tf.keras.layers.Dense(43, activation='softmax')
])

In [12]:
stack_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

file_pathV7 = './Networks/ensembles/V3/stacked_V3/StackedGeometryOpsDataAug.ckpt'

callbacksV7 = prepare_callbacks(file_pathV7)

In [13]:
print(len(labels_train), len(train_logits_preds), len(logits_train[0]))

196045 196045 196045


In [14]:
stack_model.fit(np.asarray(train_logits_preds),np.asarray(labels_train),epochs=20, batch_size=BATCH_SIZE,
                        validation_data = (np.asarray(test_logits_preds), np.asarray(labels_V2)), callbacks = callbacksV7)

Epoch 1/20
3064/3064 [==============================] - 22s 7ms/step - loss: 1.9881 - accuracy: 0.5593 - val_loss: 0.1750 - val_accuracy: 0.9811

Epoch 00001: val_accuracy improved from -inf to 0.98108, saving model to ./Networks/ensembles/V3/stacked_V3\StackedGeometryOpsDataAug.ckpt
Epoch 2/20
3064/3064 [==============================] - 21s 7ms/step - loss: 0.3760 - accuracy: 0.9292 - val_loss: 0.0532 - val_accuracy: 0.9891

Epoch 00002: val_accuracy improved from 0.98108 to 0.98907, saving model to ./Networks/ensembles/V3/stacked_V3\StackedGeometryOpsDataAug.ckpt
Epoch 3/20
3064/3064 [==============================] - 21s 7ms/step - loss: 0.1590 - accuracy: 0.9685 - val_loss: 0.0412 - val_accuracy: 0.9890

Epoch 00003: val_accuracy did not improve from 0.98907
Epoch 4/20
3064/3064 [==============================] - 21s 7ms/step - loss: 0.0991 - accuracy: 0.9782 - val_loss: 0.0430 - val_accuracy: 0.9874

Epoch 00004: val_accuracy did not improve from 0.98907
Epoch 5/20
3064/3064 [===

In [15]:
stack_model.load_weights(file_pathV7)

In [16]:
pred = stack_model.predict(np.asarray(test_logits_preds))

correct = 0

for i in range(test_dataset_length):
    if np.argmax(pred[i]) == labels_V2[i] :
        correct += 1
        
print(correct, correct/test_dataset_length)

12519 0.9912114014251782
